In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 500)

# import sys   
# sys.setrecursionlimit(25000)

from pandarallel import pandarallel

In [2]:
def extract_row(url):
    try:
        time.sleep(1)
        res=requests.get(url)
        soup = BeautifulSoup(res.content,'html.parser')
    except:
        print('cant connect',url)
        return   
    
    rows = soup.find_all('td', {'class':'html'})
    
    if len(rows):
        outputs = []
        for row in rows:
            try:
                title = row.a.text
                title = re.sub("^\s+|\s+$", "", title, flags=re.UNICODE)
                href = row.a['href']
                if href.startswith('/'):
                    href = 'https://www.refworld.org' + href

                p = soup.find_all('td', {'class':'html'})[0].p.text.split('|')
                date = re.sub("^\s+|\s+$", "", p[0], flags=re.UNICODE)
                Publisher = re.sub("^\s+|\s+$", "", p[1], flags=re.UNICODE)
                types = re.sub("^\s+|\s+$", "", p[2], flags=re.UNICODE) 

            except:
                print(url)

            output = {'title':title, 'date':date,'Publisher':Publisher,'types':types, 'href': href}
            outputs.append(output)
    else:
        outputs = None      
    return outputs

from newspaper import Article
def extract_text(url):
    try:
        time.sleep(2)
        a = Article(url)
        a.download()
        a.parse()
        text = a.text
    except:
        text = ''
    return text

# Canada

In [3]:
# prefix = 'https://www.refworld.org/publisher,USDOS,,,,,'
# suffix = '.html'

# df = pd.DataFrame(range(0, 13681,10), columns = ['page'])
# df.page = prefix + df.page.astype('str') + suffix
# df

In [4]:
# # df=pd.read_csv('US_raw.csv', header=[0])
# df1 = df.loc[df.rows.isnull()].copy().reset_index(drop=True)
# df1

In [5]:
# pandarallel.initialize(nb_workers=4)
# batch_size = 4

# i=resume=0

# while i< df1.shape[0]:

#     df1.loc[i: i+ batch_size-1, 'rows']= df1.loc[i: i+ batch_size-1, 'page'].parallel_apply(extract_row).values
    
#     df2 = df1.loc[i: i+ batch_size-1, 'rows']
#     valid_len =  df2[~df2.isnull()].shape[0]
    
#     print(i,':',valid_len, end='\t')
    
#     i += batch_size
#     if (i-resume)%(batch_size*5)==0:   
#         df1.to_csv('US_null.csv', index=False)
# df1.to_csv('US_null.csv', index=False)

In [3]:
def get_all_links(href,number):
    return [href.replace('0.html', '%s.html'%i) for i in range(0, number-number%10+1, 10)]

In [4]:
url = 'https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,,,,0.html'
res=requests.get(url, timeout=20)
soup = BeautifulSoup(res.content,'html.parser')

ul = soup.find_all('ul', class_ = 'rwbullets')[0]
country_list = ul.find_all('li')

outputs = []
for country_link in country_list:
    href = 'https://www.refworld.org'+ country_link.a['href']
    country = country_link.a.text
    number = country_link.text.split()[-1]
    number = number.replace(',', '')   # Remove period 1,233--> 1233
    number = int(number)
    output = {'country':country,'number':number,'base_href':href}
    outputs.append(output)
    
df= pd.json_normalize(outputs)
df

,country,number,base_href
0,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html"
1,Albania,448,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ALB,,,0.html"
2,Algeria,409,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,DZA,,,0.html"
3,American Samoa,1,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ASM,,,0.html"
4,Angola,289,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AGO,,,0.html"
...,...,...,...
194,Viet Nam,136,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,VNM,,,0.html"
195,Western Sahara Territory,11,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ESH,,,0.html"
196,Yemen,167,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,YEM,,,0.html"
197,Zambia,57,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZMB,,,0.html"


In [5]:
df['href']= df.apply(lambda x: get_all_links(x['base_href'], x['number']), axis=1)
df= df.explode('href').reset_index(drop=True)
df

,country,number,base_href,href
0,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html"
1,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,10.html"
2,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,20.html"
3,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,30.html"
4,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,40.html"
...,...,...,...,...
4358,Zimbabwe,204,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,160.html"
4359,Zimbabwe,204,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,170.html"
4360,Zimbabwe,204,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,180.html"
4361,Zimbabwe,204,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,190.html"


In [6]:
def extract_row(url):
    try:
        time.sleep(1)
        res=requests.get(url, timeout=20)
        soup = BeautifulSoup(res.content,'html.parser')
    except:
        print('cant connect',url)
        return   
    
    rows = soup.find_all('td', {'class':'html'})
    
    if len(rows):
        outputs = []
        for row in rows:
            try:
                title = row.a.text
                title = re.sub("^\s+|\s+$", "", title, flags=re.UNICODE)
                href = row.a['href']
                if href.startswith('/'):
                    href = 'https://www.refworld.org' + href

                p = soup.find_all('td', {'class':'html'})[0].p.text.split('|')
                date = re.sub("^\s+|\s+$", "", p[0], flags=re.UNICODE)
                Publisher = re.sub("^\s+|\s+$", "", p[1], flags=re.UNICODE)
                types = re.sub("^\s+|\s+$", "", p[2], flags=re.UNICODE) 

            except:
                print(url)

            output = {'title':title, 'date':date,'Publisher':Publisher,'types':types, 'href': href}
            outputs.append(output)
    else:
        outputs = None      
    return outputs

In [7]:
df1= df.copy()

In [8]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size = 16*10
i=0
while i < df1.shape[0]:
    print(i, end=',')
    df1.loc[i:i+batch_size-1, 'rows']=\
    df1.loc[i:i+batch_size-1, 'href'].parallel_apply(extract_row).values
    i = i + batch_size 
df1.to_csv('Canada_rows.csv', index=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,cant connect https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,NGA,,,1540.html
cant connect https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,NGA,,,1140.html
3040,3200,3360,3520,3680,3840,4000,4160,4320,

In [9]:
df1[df1.rows.isnull()]

,country,number,base_href,href,rows
154,Antigua and Barbuda,30,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ATG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ATG,,,30.html",None
233,Azerbaijan,170,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AZE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AZE,,,170.html",None
335,Belgium,40,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,BEL,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,BEL,,,40.html",None
358,Bolivia,130,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,BOL,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,BOL,,,130.html",None
371,Botswana,30,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,BWA,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,BWA,,,30.html",None
480,Cambodia,40,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,KHM,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,KHM,,,40.html",None
991,Costa Rica,290,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,CRI,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,CRI,,,290.html",None
1071,Czech Republic,180,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,CZE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,CZE,,,180.html",None
1249,Equatorial Guinea,30,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,GNQ,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,GNQ,,,30.html",None
1381,France,190,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,FRA,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,FRA,,,190.html",None


In [10]:
df1 = df1[~df1.rows.isnull()]
df1.shape

(4341, 5)

In [11]:
df1 = df1.explode('rows').reset_index(drop=True)
df1

,country,number,base_href,href,rows
0,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","{'title': 'Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018)', 'date': '13 February 2018', 'Publisher': 'Publisher: Canada: Immigration and Refugee Board of Canada', 'types': 'Document type: Query Responses', 'href': 'https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c044,0.html'}"
1,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","{'title': 'Afghanistan : information sur la loi sur les passeports du 25 octobre 2015; l'aspect et les caractéristiques de sécurité des passeports; spécimens de passeport (2015-janvier 2018)', 'date': '13 February 2018', 'Publisher': 'Publisher: Canada: Immigration and Refugee Board of Canada', 'types': 'Document type: Query Responses', 'href': 'https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c547,0.html'}"
2,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","{'title': 'Afghanistan: Treatment of members and former members of the Afghan military by the Taliban and other anti-government groups, including whether they are personally targeted outside of active conflict zones; whether rank, branch or type of military services poses a greater risk; information on state protection, including protection programs offered by the military (2012-August 2017)', 'date': '13 February 2018', 'Publisher': 'Publisher: Canada: Immigration and Refugee Board of Canad..."
3,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","{'title': 'Afghanistan : information sur le traitement réservé aux militaires et anciens militaires afghans par les talibans et d'autres groupes opposés au gouvernement, y compris la question de savoir s'ils sont ciblés personnellement à l'extérieur des zones de conflit actif; information indiquant si le rang, la branche ou le type de service militaire entraîne un risque plus élevé; information sur la protection accordée par l'État, y compris les programmes de protection offerts par l'armée ..."
4,Afghanistan,345,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,,,0.html","{'title': 'Afghanistan : information sur les exigences et la marche à suivre pour obtenir, renouveler et remplacer un passeport biométrique, à l'intérieur et à l'extérieur du pays; la période de validité du passeport; information indiquant si le lieu de naissance inscrit sur le passeport est identique à celui inscrit sur la tazkira, et s'il correspond au lieu de naissance du titulaire du passeport ou au lieu de naissance de son père (2015-janvier 2017)', 'date': '13 February 2018', 'Publishe..."
...,...,...,...,...,...
42510,Zimbabwe,204,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,190.html","{'title': 'Grèce : 1) Situation des ressortissants du Zimbabwe en Grèce. 2) Possibilité pour les ressortissants du Zimbabwe d'obtenir des permis de travail, la résidence permanente et/ou la citoyenneté en Grèce (conditions, périodes). 3) Attitude des autorités grècques envers les réfugiés du Zimbabwe', 'date': '1 April 1994', 'Publisher': 'Publisher: Canada: Immigration and Refugee Board of Canada', 'types': 'Document type: Query Responses', 'href': 'https://www.refworld.org/publisher,IRBC,Q..."
42511,Zimbabwe,204,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,0.html","https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,,,200.html","{'title': 'Zimbabwe: 1) 

In [12]:
df1[df1.rows.isnull()]

,country,number,base_href,href,rows


In [13]:
df1= df1[~df1.rows.isnull()]
df1 = pd.json_normalize(df1['rows']).reset_index(drop=True)
df1

,title,date,Publisher,types,href
0,Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018),13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c044,0.html"
1,Afghanistan : information sur la loi sur les passeports du 25 octobre 2015; l'aspect et les caractéristiques de sécurité des passeports; spécimens de passeport (2015-janvier 2018),13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c547,0.html"
2,"Afghanistan: Treatment of members and former members of the Afghan military by the Taliban and other anti-government groups, including whether they are personally targeted outside of active conflict zones; whether rank, branch or type of military services poses a greater risk; information on state protection, including protection programs offered by the military (2012-August 2017)",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa910407,0.html"
3,"Afghanistan : information sur le traitement réservé aux militaires et anciens militaires afghans par les talibans et d'autres groupes opposés au gouvernement, y compris la question de savoir s'ils sont ciblés personnellement à l'extérieur des zones de conflit actif; information indiquant si le rang, la branche ou le type de service militaire entraîne un risque plus élevé; information sur la protection accordée par l'État, y compris les programmes de protection offerts par l'armée (2012-août ...",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa910bc7,0.html"
4,"Afghanistan : information sur les exigences et la marche à suivre pour obtenir, renouveler et remplacer un passeport biométrique, à l'intérieur et à l'extérieur du pays; la période de validité du passeport; information indiquant si le lieu de naissance inscrit sur le passeport est identique à celui inscrit sur la tazkira, et s'il correspond au lieu de naissance du titulaire du passeport ou au lieu de naissance de son père (2015-janvier 2017)",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,58b42a164,0.html"
...,...,...,...,...,...
42510,"Grèce : 1) Situation des ressortissants du Zimbabwe en Grèce. 2) Possibilité pour les ressortissants du Zimbabwe d'obtenir des permis de travail, la résidence permanente et/ou la citoyenneté en Grèce (conditions, périodes). 3) Attitude des autorités grècques envers les réfugiés du Zimbabwe",1 April 1994,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,3ae6acd32c,0.html"
42511,Zimbabwe: 1) Constitutional changes in Zimbabwe in January 1990 2) Situation of whites in Zimbabwe 3) Situation of opposition in Zimbabwe,1 February 1990,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,3ae6acc048,0.html"
42512,"Jordan: 1. Status of Zimbabwean national in Jordan; 2. Can a Zimbabwean national obtain a working permit, permanent residence or citizenship status in Jordan; if so, what are the terms and conditions? 3. Jordanian authorities attitude toward or treatment of Zimbabwean refugees?",1 February 1990,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,ZWE,3ae6ab1f78,0.html"
42513,Zimbabwe: 1. Information on dissident groups in the Province of Umtali (espec

In [14]:
df1.Publisher.unique(),df1.types.unique(), df1[df1.href.duplicated()].shape

(array(['Publisher: Canada: Immigration and Refugee Board of Canada'],
       dtype=object),
 array(['Document type: Query Responses'], dtype=object),
 (0, 5))

In [15]:
pandarallel.initialize(nb_workers=8, progress_bar=False)
batch_size = 8*10

i=resume=0
while i < df1.shape[0]:
    df1.loc[i:i+batch_size-1, 'text']=\
    df1.loc[i:i+batch_size-1, 'href'].parallel_apply(extract_text).values

    print(i, end=',')
    
    if (i - resume)%(batch_size * 10) ==0:
        df1.to_csv('Canada_text.csv', index=False)
        
    i = i + batch_size
    
df1.to_csv('Canada_text.csv', index=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,80,160,240,320,400,480,560,640,720,800,880,960,1040,1120,1200,1280,1360,1440,1520,1600,1680,1760,1840,1920,2000,2080,2160,2240,2320,2400,2480,2560,2640,2720,2800,2880,2960,3040,3120,3200,3280,3360,3440,3520,3600,3680,3760,3840,3920,4000,4080,4160,4240,4320,4400,4480,4560,4640,4720,4800,4880,4960,5040,5120,5200,5280,5360,5440,5520,5600,5680,5760,5840,5920,6000,6080,6160,6240,6320,6400,6480,6560,6640,6720,6800,6880,6960,7040,7120,7200,7280,7360,7440,7520,7600,7680,7760,7840,7920,8000,8080,8160,8240,8320,8400,8480,8560,8640,8720,8800,8880,8960,9040,9120,9200,9280,9360,9440,9520,9600,9680,9760,9840,9920,10000,10080,10160,10240,10320,10400,10480,10560,10640,10720,10800,10880,10960,11040,11120,11200,11280,11360,11440,11520,11600,11680,11760,11840,11920,12000,12080,12160,12240,12320,12400,12480,12560,12640,12720,12800,12880,12960,13040,13120,13200,132

In [16]:
quit()

In [2]:
df1=pd.read_csv('Canada_text.csv', header=[0])
df1

,title,date,Publisher,types,href,text
0,Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018),13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c044,0.html","Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018)\n\nPublisher Canada: Immigration and Refugee Board of Canada Publication Date 13 February 2018 Citation / Document Symbol AFG106043.E Cite as Canada: Immigration and Refugee Board of Canada, Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018), 13 February 2018, AFG106043.E, ava..."
1,Afghanistan : information sur la loi sur les passeports du 25 octobre 2015; l'aspect et les caractéristiques de sécurité des passeports; spécimens de passeport (2015-janvier 2018),13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c547,0.html",Search Refworld\n\nand / or country All countries Afghanistan Albania Algeria American Samoa Andorra Angola Anguilla Antigua and Barbuda Argentina Armenia Aruba Australia Austria Azerbaijan Bahamas Bahrain Bangladesh Barbados Belarus Belgium Belize Benin Bermuda Bhutan Bolivia Bosnia and Herzegovina Botswana Brazil British Virgin Islands Brunei Darussalam Bulgaria Burkina Faso Burundi Cambodia Cameroon Canada Cape Verde Cayman Islands Central African Republic Chad Chile China Cocos (Keeling)...
2,"Afghanistan: Treatment of members and former members of the Afghan military by the Taliban and other anti-government groups, including whether they are personally targeted outside of active conflict zones; whether rank, branch or type of military services poses a greater risk; information on state protection, including protection programs offered by the military (2012-August 2017)",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa910407,0.html","Afghanistan: Treatment of members and former members of the Afghan military by the Taliban and other anti-government groups, including whether they are personally targeted outside of active conflict zones; whether rank, branch or type of military services poses a greater risk; information on state protection, including protection programs offered by the military (2012-August 2017)\n\nPublisher Canada: Immigration and Refugee Board of Canada Publication Date 31 August 2017 Citation / Document..."
3,"Afghanistan : information sur le traitement réservé aux militaires et anciens militaires afghans par les talibans et d'autres groupes opposés au gouvernement, y compris la question de savoir s'ils sont ciblés personnellement à l'extérieur des zones de conflit actif; information indiquant si le rang, la branche ou le type de service militaire entraîne un risque plus élevé; information sur la protection accordée par l'État, y compris les programmes de protection offerts par l'armée (2012-août ...",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa910bc7,0.html","Afghanistan : information sur le traitement réservé aux militaires et anciens militaires afghans par les talibans et d'autres groupes opposés au gouvernement, y compris la question de savoir s'ils sont ciblés personnellement à l'extérieur des zones de conflit actif; information indiquant si le rang, la branche ou le type de service militaire entraîne un risque plus élevé; information sur la protection accordée par l'État, y compris les programmes de protection offerts par l'armée (2012-août ..."
4,"Afghan

In [5]:
df1=df1[~df1.text.isnull()]

In [9]:
df1.text.str.len().describe()

count    42504.000000
mean      5386.375588
std       5948.008042
min        269.000000
25%       2500.000000
50%       3010.500000
75%       5583.000000
max      89488.000000
Name: text, dtype: float64

In [11]:
df1[df1.text.str.contains('Citation / Document Symbol')]

,title,date,Publisher,types,href,text
0,Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018),13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa91c044,0.html","Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018)\n\nPublisher Canada: Immigration and Refugee Board of Canada Publication Date 13 February 2018 Citation / Document Symbol AFG106043.E Cite as Canada: Immigration and Refugee Board of Canada, Afghanistan: Passport Law of 25 October 2015; appearance and security features of passports; specimen of passports (2015-January 2018), 13 February 2018, AFG106043.E, ava..."
2,"Afghanistan: Treatment of members and former members of the Afghan military by the Taliban and other anti-government groups, including whether they are personally targeted outside of active conflict zones; whether rank, branch or type of military services poses a greater risk; information on state protection, including protection programs offered by the military (2012-August 2017)",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa910407,0.html","Afghanistan: Treatment of members and former members of the Afghan military by the Taliban and other anti-government groups, including whether they are personally targeted outside of active conflict zones; whether rank, branch or type of military services poses a greater risk; information on state protection, including protection programs offered by the military (2012-August 2017)\n\nPublisher Canada: Immigration and Refugee Board of Canada Publication Date 31 August 2017 Citation / Document..."
3,"Afghanistan : information sur le traitement réservé aux militaires et anciens militaires afghans par les talibans et d'autres groupes opposés au gouvernement, y compris la question de savoir s'ils sont ciblés personnellement à l'extérieur des zones de conflit actif; information indiquant si le rang, la branche ou le type de service militaire entraîne un risque plus élevé; information sur la protection accordée par l'État, y compris les programmes de protection offerts par l'armée (2012-août ...",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,5aa910bc7,0.html","Afghanistan : information sur le traitement réservé aux militaires et anciens militaires afghans par les talibans et d'autres groupes opposés au gouvernement, y compris la question de savoir s'ils sont ciblés personnellement à l'extérieur des zones de conflit actif; information indiquant si le rang, la branche ou le type de service militaire entraîne un risque plus élevé; information sur la protection accordée par l'État, y compris les programmes de protection offerts par l'armée (2012-août ..."
4,"Afghanistan : information sur les exigences et la marche à suivre pour obtenir, renouveler et remplacer un passeport biométrique, à l'intérieur et à l'extérieur du pays; la période de validité du passeport; information indiquant si le lieu de naissance inscrit sur le passeport est identique à celui inscrit sur la tazkira, et s'il correspond au lieu de naissance du titulaire du passeport ou au lieu de naissance de son père (2015-janvier 2017)",13 February 2018,Publisher: Canada: Immigration and Refugee Board of Canada,Document type: Query Responses,"https://www.refworld.org/publisher,IRBC,QUERYRESPONSE,AFG,58b42a164,0.html","Afghanistan : information sur les exigences et la marche à suivre pour obtenir, renouveler et remplacer un passeport biométrique, à l'intérieur et à l'extérieur du pays; la période de validité du passeport; information indiquant si le lieu 